# Implementing Recommender Systems - Lab

## Introduction

In this lab, you'll practice creating a recommender system model using `surprise`. You'll also get the chance to create a more complete recommender system pipeline to obtain the top recommendations for a specific user.


## Objectives

In this lab you will: 

- Use surprise's built-in reader class to process data to work with recommender algorithms 
- Obtain a prediction for a specific user for a particular item 
- Introduce a new user with rating to a rating matrix and make recommendations for them 
- Create a function that will return the top n recommendations for a user 


For this lab, we will be using the famous 1M movie dataset. It contains a collection of user ratings for many different movies. In the last lesson, you were exposed to working with `surprise` datasets. In this lab, you will also go through the process of reading in a dataset into the `surprise` dataset format. To begin with, load the dataset into a Pandas DataFrame. Determine which columns are necessary for your recommendation system and drop any extraneous ones.

In [7]:
import pandas as pd
df = pd.read_csv('./ml-latest-small/ratings.csv')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100836 entries, 0 to 100835
Data columns (total 4 columns):
 #   Column     Non-Null Count   Dtype  
---  ------     --------------   -----  
 0   userId     100836 non-null  int64  
 1   movieId    100836 non-null  int64  
 2   rating     100836 non-null  float64
 3   timestamp  100836 non-null  int64  
dtypes: float64(1), int64(3)
memory usage: 3.1 MB


In [9]:
# Drop unnecessary columns
new_df = df.drop('timestamp', axis=1)

In [10]:
new_df.head()

,userId,movieId,rating
0,1,1,4.0
1,1,3,4.0
2,1,6,4.0
3,1,47,5.0
4,1,50,5.0


It's now time to transform the dataset into something compatible with `surprise`. In order to do this, you're going to need `Reader` and `Dataset` classes. There's a method in `Dataset` specifically for loading dataframes.

In [6]:
#!pip install scikit-surprise

In [119]:
from surprise import Reader, Dataset
# read in values as Surprise dataset 
reader = Reader()
data = Dataset.load_from_df(new_df, reader)

Let's look at how many users and items we have in our dataset. If using neighborhood-based methods, this will help us determine whether or not we should perform user-user or item-item similarity

In [13]:
dataset = data.build_full_trainset()
print('Number of users: ', dataset.n_users, '\n')
print('Number of items: ', dataset.n_items)

Number of users:  610 

Number of items:  9724


## Determine the best model 

Now, compare the different models and see which ones perform best. For consistency sake, use RMSE to evaluate models. Remember to cross-validate! Can you get a model with a higher average RMSE on test data than 0.869?

In [14]:
# importing relevant libraries
from surprise.model_selection import cross_validate
from surprise.prediction_algorithms import SVD
from surprise.prediction_algorithms import KNNWithMeans, KNNBasic, KNNBaseline
from surprise.model_selection import GridSearchCV
import numpy as np

In [15]:
## Perform a gridsearch with SVD
# ⏰ This cell may take several minutes to run

param_grid = {'n_factors':[20, 50, 100], 'reg_all': [0.02, 0.05, 0.1]}
gs_model = GridSearchCV(SVD, param_grid=param_grid, n_jobs = -1, joblib_verbose=5)
gs_model.fit(data)

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  10 tasks      | elapsed:   15.0s
[Parallel(n_jobs=-1)]: Done  45 out of  45 | elapsed:  1.5min finished


In [20]:
# print out optimal parameters for SVD after GridSearch
print('Best score:', gs_model.best_score)
print('Best parameters:', gs_model.best_params)

Best score: {'rmse': 0.8687182695413209, 'mae': 0.6677087971135196}
Best parameters: {'rmse': {'n_factors': 20, 'reg_all': 0.02}, 'mae': {'n_factors': 20, 'reg_all': 0.02}}


In [22]:
# cross validating with KNNBasic
knn_basic = KNNBasic(sim_options={'name':'pearson', 'user_based':True})
cv_knn_basic = cross_validate(knn_basic, data, n_jobs=-1)

In [32]:
# print out the average RMSE score for the test set
print('RMSE scores:', cv_knn_basic['test_rmse'])
print('Avg RMSE score:', np.mean(cv_knn_basic['test_rmse']))

RMSE scores: [0.98002504 0.9652961  0.97961854 0.96673369 0.96705843]
Avg RMSE score: 0.9717463620206681


In [33]:
# cross validating with KNNBaseline
knn_baseline = KNNBaseline(sim_options={'name':'pearson', 'user_based':True})
cv_knn_baseline = cross_validate(knn_basic, data, n_jobs=-1)

In [35]:
# print out the average score for the test set
print('Avg RMSE score:', np.mean(cv_knn_baseline['test_rmse']))

Avg RMSE score: 0.972779300722134


Based off these outputs, it seems like the best performing model is the SVD model with `n_factors = 50` and a regularization rate of 0.05. Use that model or if you found one that performs better, feel free to use that to make some predictions.

## Making Recommendations

It's important that the output for the recommendation is interpretable to people. Rather than returning the `movie_id` values, it would be far more valuable to return the actual title of the movie. As a first step, let's read in the movies to a dataframe and take a peek at what information we have about them.

In [36]:
df_movies = pd.read_csv('./ml-latest-small/movies.csv')

In [37]:
df_movies.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


## Making simple predictions
Just as a reminder, let's look at how you make a prediction for an individual user and item. First, we'll fit the SVD model we had from before.

In [58]:
#svd = SVD(n_factors= 50, reg_all=0.05)  
svd = SVD(n_factors= 20, reg_all=0.02)  #Best paremeters according to the grid search
svd.fit(dataset)

In [57]:
cv_svd = cross_validate(svd, data, n_jobs=-1)
print('Avg RMSE score:', np.mean(cv_svd['test_rmse']))

Avg RMSE score: 0.869904358207387


In [59]:
svd.predict(2, 4)

Prediction(uid=2, iid=4, r_ui=None, est=3.0770973342817625, details={'was_impossible': False})

This prediction value is a tuple and each of the values within it can be accessed by way of indexing. Now let's put our knowledge of recommendation systems to do something interesting: making predictions for a new user!

## Obtaining User Ratings 

It's great that we have working models and everything, but wouldn't it be nice to get to recommendations specifically tailored to your preferences? That's what we'll be doing now. The first step is to create a function that allows us to pick randomly selected movies. The function should present users with a movie and ask them to rate it. If they have not seen the movie, they should be able to skip rating it. 

The function `movie_rater()` should take as parameters: 

* `movie_df`: DataFrame - a dataframe containing the movie ids, name of movie, and genres
* `num`: int - number of ratings
* `genre`: string - a specific genre from which to draw movies

The function returns:
* rating_list : list - a collection of dictionaries in the format of {'userId': int , 'movieId': int , 'rating': float}

#### This function is optional, but fun :) 

In [127]:
def movie_rater(movie_df, num, genre=None):
    
    userID = 999
    
    movies_list = [] 
    
    for i in range(0,num):
        if genre:
            movie = movie_df[movie_df['genres'].str.contains(genre)].sample(1)
        else:
            movie = movie_df.sample(1)
        

        print(movie.title)   
        rating = input("How do you rate this movie? Press 1-5 to rate. If you have not seen this movie, press 'n'.")
        print('\n')
        
        if rating == 'n':
            continue
        else:
            movies_list.append({'userId': userID , 'movieId': movie.movieId.values[0] , 'rating': rating})
    
    return movies_list

In [133]:
# try out the new function here!
my_ratings = movie_rater(df_movies, 10, 'Fantasy')

9194    The Love Bug (1997)
Name: title, dtype: object
How do you rate this movie? Press 1-5 to rate. If you have not seen this movie, press 'n'.n


169    Three Wishes (1995)
Name: title, dtype: object
How do you rate this movie? Press 1-5 to rate. If you have not seen this movie, press 'n'.n


6629    Golden Compass, The (2007)
Name: title, dtype: object
How do you rate this movie? Press 1-5 to rate. If you have not seen this movie, press 'n'.4


4137    Lord of the Rings: The Two Towers, The (2002)
Name: title, dtype: object
How do you rate this movie? Press 1-5 to rate. If you have not seen this movie, press 'n'.5


723    Ghost and Mrs. Muir, The (1947)
Name: title, dtype: object
How do you rate this movie? Press 1-5 to rate. If you have not seen this movie, press 'n'.n


7133    Cloudy with a Chance of Meatballs (2009)
Name: title, dtype: object
How do you rate this movie? Press 1-5 to rate. If you have not seen this movie, press 'n'.4


9693    Fullmetal Alchemist 2018 (2017)
Na

In [134]:
my_ratings

[{'userId': 999, 'movieId': 56171, 'rating': '4'},
 {'userId': 999, 'movieId': 5952, 'rating': '5'},
 {'userId': 999, 'movieId': 71264, 'rating': '4'},
 {'userId': 999, 'movieId': 36401, 'rating': '4'}]

In [182]:
my_ratings2 = movie_rater(df_movies, 10, 'Action')
my_ratings3 = movie_rater(df_movies, 20, 'Romance')
my_ratings4 = movie_rater(df_movies, 10, 'Sci-Fi')

9595    The Hitman's Bodyguard (2017)
Name: title, dtype: object
How do you rate this movie? Press 1-5 to rate. If you have not seen this movie, press 'n'.n


8259    Family, The (2013)
Name: title, dtype: object
How do you rate this movie? Press 1-5 to rate. If you have not seen this movie, press 'n'.n


2544    Night to Remember, A (1958)
Name: title, dtype: object
How do you rate this movie? Press 1-5 to rate. If you have not seen this movie, press 'n'.n


3355    They Live (1988)
Name: title, dtype: object
How do you rate this movie? Press 1-5 to rate. If you have not seen this movie, press 'n'.n


3854    Bourne Identity, The (2002)
Name: title, dtype: object
How do you rate this movie? Press 1-5 to rate. If you have not seen this movie, press 'n'.5


793    Die Hard (1988)
Name: title, dtype: object
How do you rate this movie? Press 1-5 to rate. If you have not seen this movie, press 'n'.4


9547    Final Flight of the Osiris (2003)
Name: title, dtype: object
How do you rate this

In [183]:
my_ratings5 = movie_rater(df_movies, 10, 'Adventure')
my_ratings6 = movie_rater(df_movies, 10, 'Drama') 
my_ratings7 = movie_rater(df_movies, 10, 'Comedy')

5977    Valiant (2005)
Name: title, dtype: object
How do you rate this movie? Press 1-5 to rate. If you have not seen this movie, press 'n'.n


9437    Passengers (2016)
Name: title, dtype: object
How do you rate this movie? Press 1-5 to rate. If you have not seen this movie, press 'n'.n


5330    Sky Captain and the World of Tomorrow (2004)
Name: title, dtype: object
How do you rate this movie? Press 1-5 to rate. If you have not seen this movie, press 'n'.5


7691    Phineas and Ferb the Movie: Across the 2nd Dim...
Name: title, dtype: object
How do you rate this movie? Press 1-5 to rate. If you have not seen this movie, press 'n'.n


3666    Snow Dogs (2002)
Name: title, dtype: object
How do you rate this movie? Press 1-5 to rate. If you have not seen this movie, press 'n'.n


8489    Jupiter Ascending (2015)
Name: title, dtype: object
How do you rate this movie? Press 1-5 to rate. If you have not seen this movie, press 'n'.4


9535    Despicable Me 3 (2017)
Name: title, dtype: objec

If you're struggling to come up with the above function, you can use this list of user ratings to complete the next segment

### Making Predictions With the New Ratings
Now that you have new ratings, you can use them to make predictions for this new user. The proper way this should work is:

* add the new ratings to the original ratings DataFrame, read into a surprise dataset
* train a model using the new combined DataFrame
* make predictions for the user
* order those predictions from highest rated to lowest rated
* return the top n recommendations with the text of the actual movie (rather than just the index number)

In [191]:
ratings = my_ratings + my_ratings2 + my_ratings3 + my_ratings4 + my_ratings5 + my_ratings6 + my_ratings7
ratings

[{'userId': 999, 'movieId': 56171, 'rating': '4'},
 {'userId': 999, 'movieId': 5952, 'rating': '5'},
 {'userId': 999, 'movieId': 71264, 'rating': '4'},
 {'userId': 999, 'movieId': 36401, 'rating': '4'},
 {'userId': 999, 'movieId': 5418, 'rating': '5'},
 {'userId': 999, 'movieId': 1036, 'rating': '4'},
 {'userId': 999, 'movieId': 933, 'rating': '4'},
 {'userId': 999, 'movieId': 77866, 'rating': '5'},
 {'userId': 999, 'movieId': 74154, 'rating': '4'},
 {'userId': 999, 'movieId': 72921, 'rating': '3'},
 {'userId': 999, 'movieId': 4308, 'rating': '4'},
 {'userId': 999, 'movieId': 68358, 'rating': '5'},
 {'userId': 999, 'movieId': 8865, 'rating': '5'},
 {'userId': 999, 'movieId': 113345, 'rating': '4'},
 {'userId': 999, 'movieId': 172547, 'rating': '5'},
 {'userId': 999, 'movieId': 1376, 'rating': '4'},
 {'userId': 999, 'movieId': 41566, 'rating': '5'},
 {'userId': 999, 'movieId': 594, 'rating': '4'},
 {'userId': 999, 'movieId': 5480, 'rating': '3'},
 {'userId': 999, 'movieId': 1247, 'ratin

In [194]:
## add the new ratings to the original ratings DataFrame
new_df_ratings = new_df.append(ratings, ignore_index=True)
new_df_ratings.tail(10)

,userId,movieId,rating
100846,999,4308,4
100847,999,68358,5
100848,999,8865,5
100849,999,113345,4
100850,999,172547,5
100851,999,1376,4
100852,999,41566,5
100853,999,594,4
100854,999,5480,3
100855,999,1247,4


In [195]:
data_rt = Dataset.load_from_df(new_df_ratings, reader)
dataset_rt = data_rt.build_full_trainset()

In [196]:
# train a model using the new combined DataFrame

svd_rt = SVD(n_factors= 20, reg_all=0.02)  
svd_rt.fit(dataset_rt)

In [197]:
pred = svd_rt.predict(999, 3455)[3]
pred

3.905750923200167

In [199]:
movieID_list = new_df_ratings['movieId'].unique()
#list(movieID_list)

In [200]:
# make predictions for the user
# you'll probably want to create a list of tuples in the format (movie_id, predicted_score)

movieID_list = new_df_ratings['movieId'].unique()
predictions_forUser = []

for ID in movieID_list:
    pred = svd_rt.predict(999, ID)[3]
    predictions_forUser.append((ID,pred))
    
predictions_forUser[0:5]

[(1, 4.380860416692458),
 (3, 3.8621417855679616),
 (6, 4.575849297639396),
 (47, 4.6247387824067125),
 (50, 4.833195607173474)]

In [201]:
# order the predictions from highest to lowest rated
ranked_movies = sorted(predictions_forUser, key=lambda x:x[1], reverse=True)
ranked_movies[0:5]

[(318, 5),
 (750, 4.948509321670439),
 (1221, 4.927438902993958),
 (858, 4.91741861523349),
 (1204, 4.902927943750866)]

In [202]:
ranked_movies[3][0]

858

 For the final component of this challenge, it could be useful to create a function `recommended_movies()` that takes in the parameters:
* `user_ratings`: list - list of tuples formulated as (user_id, movie_id) (should be in order of best to worst for this individual)
* `movie_title_df`: DataFrame 
* `n`: int - number of recommended movies 

The function should use a `for` loop to print out each recommended *n* movies in order from best to worst

In [203]:
# return the top n recommendations using the 
def recommended_movies(user_ratings,movie_title_df,n):
        
        for i in range(0,n):
            ID = user_ratings[i][0]
            movie = movie_title_df[movie_title_df.movieId == ID]
            print(movie.title)
            
recommended_movies(ranked_movies,df_movies,5)

277    Shawshank Redemption, The (1994)
Name: title, dtype: object
602    Dr. Strangelove or: How I Learned to Stop Worr...
Name: title, dtype: object
922    Godfather: Part II, The (1974)
Name: title, dtype: object
659    Godfather, The (1972)
Name: title, dtype: object
906    Lawrence of Arabia (1962)
Name: title, dtype: object


## Level Up (Optional)

* Try and chain all of the steps together into one function that asks users for ratings for a certain number of movies, then all of the above steps are performed to return the top $n$ recommendations
* Make a recommender system that only returns items that come from a specified genre

## Summary

In this lab, you got the chance to implement a collaborative filtering model as well as retrieve recommendations from that model. You also got the opportunity to add your own recommendations to the system to get new recommendations for yourself! Next, you will learn how to use Spark to make recommender systems.